# **Review Analysis**
Term paper: Determinants of Moscow Public Catering Establishments' Consumer Evaluations

In [ ]:
import pandas as pd
!pip install pymorphy2==0.9.1
import pymorphy2
import inspect
import re
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=8a4c2d0e1909b783599582654ce387438a32700292fa30ce3751f304e7cec299
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
 data = pd.read_csv('/content/yandex_with_coordinates.csv')

In [ ]:
reviews = data.copy()

In [ ]:
morph = pymorphy2.MorphAnalyzer(lang='ru')

nltk.download('stopwords')
stops = nltk.corpus.stopwords.words('russian')

stops.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на', 'руб', 'мой', 'твой', 'его', 'её', 'наш', 'ваш', 'их', 'свой', 'еще', 'очень', 'поэтому', 'однако', 'конечно'])
unique_stops = set(stops)

def extract_nouns(text):
    nouns = []

    clean_text = re.sub(r'\s+', ' ', re.sub(r'[\d\W]', ' ', text))

    words = clean_text.split()

    for word in words:
        parsed_word = morph.parse(word)[0]

        normalized_word = parsed_word.normal_form
        if normalized_word not in unique_stops:

            pos = parsed_word.tag.POS
            case = parsed_word.tag.case
            anim = parsed_word.tag.animacy

            if pos == 'NOUN' and not (case == 'nomn' and anim == 'anim'):
                nouns.append(normalized_word)

    return ' '.join(nouns)

reviews['aspects'] = reviews['review'].apply(extract_nouns)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
reviews['category_one'] = reviews['category'].str.split(', ').str[0]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

stop_words = ["ресторан", "тортик", "заведение", "пиццерия", "кафе", "столовая", "магазин", "пекарня", "кофейня", "место", "день", "работа", "киоск", "сироп"]

def find_top_words_by_rubric(vectorizer, reviews):

    result = {
        'category_one': [],
        'words': [],
        'reviews': [],
        'scores': []
    }

    for rubric in reviews['category_one'].unique():
        texts = reviews[reviews['category_one'] == rubric]['aspects']
        total_count = texts.shape[0]

        if total_count >= 30:
            tfidf_matrix = vectorizer.fit_transform(texts)
        else:
            continue

        result['category_one'].append(rubric)
        result['reviews'].append(total_count)
        feature_names = vectorizer.get_feature_names_out()
        tfidf_scores = tfidf_matrix.max(axis=0).toarray().ravel()

        top_words_indices = tfidf_scores.argsort()[-5:][::-1]
        top_words = [feature_names[i] for i in top_words_indices if feature_names[i] not in stop_words]
        result['words'].append(', '.join(top_words))
        top_scores = [str(tfidf_scores[i]) for i in top_words_indices if feature_names[i] not in stop_words]
        result['scores'].append(', '.join(top_scores))

    return result

aspects_vectorizer = TfidfVectorizer(use_idf=True, max_df=0.8, min_df=0.1, stop_words=stop_words)

tf_idf_aspects = pd.DataFrame(find_top_words_by_rubric(aspects_vectorizer, reviews)).sort_values(by='reviews', ascending=False)

In [ ]:
categories = ['Ресторан',
              'Кафе',
              'Бар, паб',
              'Бар',
              'Быстрое питание',
              'Пиццерия',
              'Кофейня',
              'Суши-бар',
              'Столовая',
              'Пекарня',
              'Кальян-бар',
              'Кондитерская',
              'Кофе с собой',
              'Мороженое',
              'Спортбар',
              'Безалкогольный бар',
              'Бары и клубы',
              'Винный бар']

In [ ]:
tf_idf_aspects['category_one'] = tf_idf_aspects['category_one'].astype(str)

In [ ]:
tf_idf_aspects2 = tf_idf_aspects[tf_idf_aspects['category_one'].isin(categories)]

In [ ]:
tf_idf_aspects2 = tf_idf_aspects2.drop(columns=['scores'])
tf_idf_aspects2.reset_index(drop=True, inplace=True)

In [ ]:
tf_idf_aspects2

,category_one,words,reviews
0,Ресторан,"цена, персонал, обслуживание, музыка, меню",9503
1,Кафе,"цена, персонал, обслуживание, меню, кухня",5142
2,Бар,"цена, пиво, персонал, обслуживание, напиток",1925
3,Быстрое питание,"шаурм, цена, персонал, заказ, еда",1718
4,Кофейня,"цена, персонал, напиток, кофе, еда",1520
5,Кальян-бар,"чай, персонал, обслуживание, музыка, кухня",784
6,Пиццерия,"цена, пицца, персонал, обслуживание, меню",713
7,Суши-бар,"цена, ролл, персонал, обслуживание, меню",485
8,Пекарня,"цена, хлеб, пирог, персонал, кофе",459
9,Кондитерская,"цена, торт, пирожное, персонал, кофе",361


In [ ]:
import plotly.express as px
from collections import Counter

all_words = tf_idf_aspects2['words'].str.split(', ')

all_words_list = [word for sublist in all_words for word in sublist]

word_counts = Counter(all_words_list)

word_counts_df = pd.DataFrame(list(word_counts.items()), columns=['word', 'count'])

word_counts_df = word_counts_df.sort_values(by='count', ascending=False)

fig = px.bar(word_counts_df, x='word', y='count', title='Количество упоминаний уникальных слов',
             category_orders={"word": word_counts_df['word'].tolist()},  text='count')
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
df = reviews[['rating', 'review']]

In [ ]:
df = df[df['rating'] != 0].reset_index(drop=True)

In [ ]:
from nltk.corpus import stopwords
from pymystem3 import Mystem

# Инициализация Mystem
mystem = Mystem()

# Функция для удаления стоп-слов и лемматизации текста
def preprocess_text(text):
    russian_stopwords = stopwords.words("russian")
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token.strip() and token.strip() not in russian_stopwords or token.strip() == "не"]
    return " ".join(tokens)

df['cleaned_review'] = df['review'].apply(preprocess_text)

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
import re
import nltk
from nltk.corpus import stopwords

def replace_phrase(text):
    new_text = re.sub(r'не очень (\w+)', lambda match: 'ужасно', text)
    return new_text

def new_reviews(text):
    negative_keywords = {'мало', 'не вкусные', 'не вкусные', 'не вкусное', 'не пойду'}

    new_clean_reviews = re.sub(r'[^\w\s]', '', text)
    new_clean_reviews = re.sub(r'\d+', '', new_clean_reviews)
    new_clean_reviews = re.sub(r'\s+', ' ', new_clean_reviews)

    words = new_clean_reviews.lower().split()

    nltk.download('stopwords')
    stop_words = set(stopwords.words('russian'))

    words = [word for word in words if word not in stop_words or word == 'не']

    words = replace_phrase(' '.join(words)).split()

    for word in words:
        if word in negative_keywords:
            return 'ужасно ' + ' '.join(words)

    return ' '.join(words)

df['cleaned_review'] = df['cleaned_review'].apply(new_reviews)
df['cleaned_review'] = df['cleaned_review'].str.replace(r'\b(ужасно)\s+\1\b', r'\1', regex=True)

Выходные данные были обрезаны до нескольких последних строк (5000).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is 

In [ ]:
# Функция для очистки текста от ненужных слов
def clean_text(text):
    # Удаление английских букв
    cleaned_text = re.sub(r'[a-zA-Z]', '', text)
    # Удаление однобуквенных слов
    cleaned_text = re.sub(r'\b\w\b', '', cleaned_text)
    # Удаление числовых обозначений
    cleaned_text = re.sub(r'\b(один|два|оба|три|четыре|пять|шесть|семь|восемь|девять|десять|сто|тысяча|миллион|миллиард|триллион)\b', '', cleaned_text)
    # Удаление слов "несколько" и "столько"
    cleaned_text = re.sub(r'\b(несколько|столько)\b', '', cleaned_text)
    return cleaned_text

# Применение очистки к каждому отзыву в датасете
df['cleaned_review'] = df['cleaned_review'].apply(clean_text)

In [ ]:
df

,rating,review,cleaned_review
0,5,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...",пить кофе рим париж вкусный капучий фундучный ...
1,4,"Не очень удобное расположение, от метро идти м...",ужасно расположение метро идти мина быстрый ша...
2,5,Интересное солнечное место по пути. Желтенькое...,интересный солнечный место путь желтенький люб...
3,5,Уютное ламповое место. Мне всё понравилось: ра...,уютный ламповый место понравиться расположение...
4,1,Зашли единожды в это кафе. Заказали с собой ед...,зайти единожды кафе заказывать еда прощать кар...
...,...,...,...
25813,5,"Классное заведение,интересный интерьер и вкусн...",классный заведение интересный интерьер вкусный...
25814,1,"Залетели ночью с друзьями, разбудили работника...",залетать ночь друг разбудить работник сделать ...
25815,5,Очень вкусная пицца. Нравится их система лояль...,очень вкусный пицца нравиться система лояльнос...
25816,5,"Пицца детям понравилась, интерьер очень интере...",пицца ребенок понравиться интерьер очень интер...


In [ ]:
!pip install dostoevsky

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227135 sha256=1dd0a586484fa014ec614dc848985ebc979effce11d95dda6bba4f60b0b20d51
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
!python3 -m dostoevsky download fasttext-social-network-model

In [ ]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
from nltk import sent_tokenize
import nltk

nltk.download("punkt")

tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

df['cleaned_review'] = df['cleaned_review'].astype(str)

sentences = [sent_tokenize(review) for review in df['cleaned_review']]

sentences = [sentence for sublist in sentences for sentence in sublist]

results = model.predict(sentences, k=5)

df['neutral'] = [result.get('neutral') for result in results]
df['positive'] = [result.get('positive') for result in results]
df['negative'] = [result.get('negative') for result in results]
df['skip'] = [result.get('skip') for result in results]
df['speech'] = [result.get('speech') for result in results]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df

,rating,review,cleaned_review,neutral,positive,negative,skip,speech
0,5,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...",пить кофе рим париж вкусный капучий фундучный ...,0.160276,0.334599,0.206904,0.087574,0.007131
1,4,"Не очень удобное расположение, от метро идти м...",ужасно расположение метро идти мина быстрый ша...,0.201823,0.468801,0.090103,0.023699,0.011342
2,5,Интересное солнечное место по пути. Желтенькое...,интересный солнечный место путь желтенький люб...,0.100889,0.644235,0.036230,0.060097,0.018557
3,5,Уютное ламповое место. Мне всё понравилось: ра...,уютный ламповый место понравиться расположение...,0.348655,0.600198,0.038476,0.050341,0.001144
4,1,Зашли единожды в это кафе. Заказали с собой ед...,зайти единожды кафе заказывать еда прощать кар...,0.569863,0.069552,0.294225,0.140346,0.003493
...,...,...,...,...,...,...,...,...
25813,5,"Классное заведение,интересный интерьер и вкусн...",классный заведение интересный интерьер вкусный...,0.053413,0.870607,0.015435,0.011697,0.022987
25814,1,"Залетели ночью с друзьями, разбудили работника...",залетать ночь друг разбудить работник сделать ...,0.731069,0.051855,0.122533,0.156115,0.005921
25815,5,Очень вкусная пицца. Нравится их система лояль...,очень вкусный пицца нравиться система лояльнос...,0.024433,0.956644,0.034110,0.029322,0.008587
25816,5,"Пицца детям понравилась, интерьер очень интере...",пицца ребенок понравиться интерьер очень интер...,0.152042,0.943358,0.028446,0.010663,0.003085


In [ ]:
df['review'][4]

'Зашли единожды в это кафе. Заказали с собой еду. Простите ,но так себе. Картошка фри та же самая никакая. В фаст-фуд кортах и то лучше в разы. Паста тоже ( Будто макароны по-флоцки). Вдобавок дома вскрыв коробки обнаружили, что вместо двух разных пицц нам положили две одинаковых.  😖Пиццей в принципе это можно назвать с натяжкой.Будто дешевой ветчины сверху накидали. Вся еда пресная.  Самое отвратительно место на районе. Не советую к посещению.  П.с. На негативные отзывы ставят негативные лайки явно не посетители этого «кафе».'

In [ ]:
rating_counts = df['rating'].value_counts().reset_index()
rating_counts.columns = ['Рейтинг', 'Частота']

fig = px.bar(rating_counts, x='Рейтинг', y='Частота', title='Распределение рейтингов по частоте', text = 'Частота')
fig.update_traces(textposition='outside')
fig.show()

In [ ]:
fig = px.histogram(df, x=["negative", "neutral", "positive"],
                   title='Распределение тональностей отзывов',
                   labels={'value': 'Рейтинг', 'variable': 'Тональность', 'count': 'Частота'},
                   color_discrete_map={'negative': '#445c6d', 'neutral': '#879eb0', 'positive': '#b7d6d5'})

fig.show()

In [ ]:
grouped = df.groupby('rating').apply(lambda x: pd.Series({
    'neutral > positive & negative': sum((x['neutral'] > x['positive']) & (x['neutral'] > x['negative'])),
    'positive > neutral & negative': sum((x['positive'] > x['neutral']) & (x['positive'] > x['negative'])),
    'negative > neutral & positive': sum((x['negative'] > x['neutral']) & (x['negative'] > x['positive']))
}))
grouped

,neutral > positive & negative,positive > neutral & negative,negative > neutral & positive
rating,,,
1,1316,102,468
2,567,122,197
3,711,342,254
4,843,1139,246
5,3687,15270,405


In [ ]:
grouped = grouped.reset_index()

fig = px.bar(grouped, x='rating', y=['neutral > positive & negative', 'positive > neutral & negative', 'negative > neutral & positive'],
             barmode='group', labels={'value': 'Частота', 'rating': 'Рейтинг', 'variable': 'Условие'})
fig.update_layout(title='Частота тональностей по рейтингам')

fig.show()

In [ ]:
df['tone'] = df[['negative', 'positive', 'neutral']].idxmax(axis=1)

In [ ]:
df

,rating,review,cleaned_review,neutral,positive,negative,skip,speech,tone
0,5,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...",пить кофе рим париж вкусный капучий фундучный ...,0.160276,0.334599,0.206904,0.087574,0.007131,positive
1,4,"Не очень удобное расположение, от метро идти м...",ужасно расположение метро идти мина быстрый ша...,0.201823,0.468801,0.090103,0.023699,0.011342,positive
2,5,Интересное солнечное место по пути. Желтенькое...,интересный солнечный место путь желтенький люб...,0.100889,0.644235,0.036230,0.060097,0.018557,positive
3,5,Уютное ламповое место. Мне всё понравилось: ра...,уютный ламповый место понравиться расположение...,0.348655,0.600198,0.038476,0.050341,0.001144,positive
4,1,Зашли единожды в это кафе. Заказали с собой ед...,зайти единожды кафе заказывать еда прощать кар...,0.569863,0.069552,0.294225,0.140346,0.003493,neutral
...,...,...,...,...,...,...,...,...,...
25813,5,"Классное заведение,интересный интерьер и вкусн...",классный заведение интересный интерьер вкусный...,0.053413,0.870607,0.015435,0.011697,0.022987,positive
25814,1,"Залетели ночью с друзьями, разбудили работника...",залетать ночь друг разбудить работник сделать ...,0.731069,0.051855,0.122533,0.156115,0.005921,neutral
25815,5,Очень вкусная пицца. Нравится их система лояль...,очень вкусный пицца нравиться система лояльнос...,0.024433,0.956644,0.034110,0.029322,0.008587,positive
25816,5,"Пицца детям понравилась, интерьер очень интере...",пицца ребенок понравиться интерьер очень интер...,0.152042,0.943358,0.028446,0.010663,0.003085,positive


In [ ]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('russian'))
texts = df['review'].tolist()

# Список слов для исключения из текста перед обучением LDA
stop_words_custom = ['это', 'очень', 'ресторан', 'который', 'клуб', 'салат', 'место', 'бар', 'вкусный', 'заведение', 'бургер', 'матч', 'ребенок', 'рамена', 'просто', 'заказывать', 'пиво', 'заказ', 'весь', 'человек', 'спасибо', 'ужасно', 'вкусно', 'соус']

def preprocess_text(text):
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление пунктуации
    text = re.sub(r'\W', ' ', text)
    # Токенизация
    tokens = word_tokenize(text, language='russian')
    # Удаление стоп-слов и слов из списка stop_words_custom
    tokens = [word for word in tokens if word not in stop_words and word not in stop_words_custom]
    return tokens

# Подготовка данных для LDA
processed_texts = [preprocess_text(text) for text in texts]
dictionary = corpora.Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

# Обучение модели LDA
num_topics = 11
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary)

topics_distribution = [lda_model.get_document_topics(text) for text in corpus]

topic_counts = {f'Topic {i}': 0 for i in range(num_topics)}
for doc_topics in topics_distribution:
    for topic, _ in doc_topics:
        topic_counts[f'Topic {topic}'] += 1

topic_counts_df = pd.DataFrame(list(topic_counts.items()), columns=['Тема', 'Частота'])

top_topics = topic_counts_df.sort_values(by='Частота', ascending=False)
print(top_topics)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


        Тема  Частота
7    Topic 7    21447
0    Topic 0    14528
6    Topic 6    12002
2    Topic 2    11529
1    Topic 1    10626
8    Topic 8    10306
4    Topic 4    10126
9    Topic 9    10017
3    Topic 3     9745
5    Topic 5     9512
10  Topic 10     8959


In [ ]:
top_words_per_topic = lda_model.show_topics(num_words=1)

# Вывод наиболее вероятных слов для каждой темы
for i, topic_words in top_words_per_topic:
    print(f"Тема {i}: {topic_words}")

Тема 7: 0.024*"персонал"
Тема 5: 0.004*"меню"
Тема 3: 0.006*"2"
Тема 0: 0.009*"кофе"
Тема 4: 0.011*"цены"
Тема 1: 0.008*"ещё"
Тема 10: 0.008*"заранее"
Тема 9: 0.038*"кофе"
Тема 8: 0.006*"пиццу"
Тема 6: 0.005*"отлично"


In [ ]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('russian'))
texts = df['cleaned_review'].tolist()

# Задание ключевых слов для каждой темы
topics_keywords = {
    'персонал': ['официанты', 'официант', 'работник', 'работники', 'персонал', 'кассир', 'администратор', 'хостес', 'сотрудники', 'бармен', 'бармэн', 'бариста', 'сомелье', 'обслуживание', 'сервис', 'отношение'],
    'меню': ['ассортимент', 'меню', 'выбор', 'карта', 'разнообразие', 'разнообразный'],
    'цена': ['стоимость', 'цена', 'дорого', 'дешево', 'дёшево', 'бюджетно', 'дешевый', 'бюджетный', 'дорогой'],
    'маркетинг': ['реклама', 'маркетинг', 'продвижение', 'бренд', 'акция', 'известный', 'лояльность', 'раскрученный', 'популярный', 'пиар', 'распиаренный', 'блогер', 'малоизвесный'],
    'санитария': ['санитария', 'гигиена', 'чистота', 'уборщик', 'чистый', 'грязный', 'уборка', 'убрано', 'грязно', 'пыльно', 'убрано', 'тараканы', 'волос', 'убранный', 'налет', 'мытое', 'мытый', 'немытое', 'мухи'],
    'месторасположение': ['локация', 'месторасположение', 'расположение', 'доступность', 'достопримечательность', 'удобный', 'дорога', 'рядом', 'далеко', 'неудобно', 'удобно', 'близко', 'близкий', 'далекий'],
    'атмосфера': ['обстановка', 'концепция', 'стиль', 'атмосфера', 'тематика', 'уют', 'уютный', 'вайб', 'атмосферный', 'домашний', 'дом', 'неуютный', 'неуютно', 'приятный', 'комфортный', 'комфортно', 'комфорт', 'неприятный'],
    'еда': ['сочетание', 'божественно', 'еда', 'блюдо', 'кухня', 'вкус', 'повар', 'качество', 'свежесть', 'продукт', 'качественный', 'свежий', 'вкусно', 'вкусный', 'невкусный', 'божественный']
}

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    tokens = word_tokenize(text, language='russian')
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

# Создание списка документов, в которых встречаются ключевые слова каждой темы
topic_texts = {topic: [] for topic in topics_keywords.keys()}
for text in texts:
    for topic, keywords in topics_keywords.items():
        if any(keyword in text for keyword in keywords):
            topic_texts[topic].append(text)
            break

# Подготовка данных для LDA
processed_texts = [preprocess_text(text) for sublist in topic_texts.values() for text in sublist]
dictionary = corpora.Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

# Обучение модели LDA
num_topics = len(topics_keywords)
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary)

topics_distribution = [lda_model.get_document_topics(text) for text in corpus]

topic_counts = {topic: len(topic_texts[topic]) for topic in topics_keywords.keys()}

topic_counts_df = pd.DataFrame(list(topic_counts.items()), columns=['Тема', 'Частота'])

top_topics = topic_counts_df.sort_values(by='Частота', ascending=False)
top_topics


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Тема,Частота
0,персонал,13123
7,еда,2954
1,меню,2637
6,атмосфера,2442
2,цена,1902
5,месторасположение,794
4,санитария,246
3,маркетинг,157


In [ ]:
df

,rating,review,cleaned_review,neutral,positive,negative,skip,speech,tone
0,5,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...",пить кофе рим париж вкусный капучий фундучный ...,0.160276,0.334599,0.206904,0.087574,0.007131,positive
1,4,"Не очень удобное расположение, от метро идти м...",ужасно расположение метро идти мина быстрый ша...,0.201823,0.468801,0.090103,0.023699,0.011342,positive
2,5,Интересное солнечное место по пути. Желтенькое...,интересный солнечный место путь желтенький люб...,0.100889,0.644235,0.036230,0.060097,0.018557,positive
3,5,Уютное ламповое место. Мне всё понравилось: ра...,уютный ламповый место понравиться расположение...,0.348655,0.600198,0.038476,0.050341,0.001144,positive
4,1,Зашли единожды в это кафе. Заказали с собой ед...,зайти единожды кафе заказывать еда прощать кар...,0.569863,0.069552,0.294225,0.140346,0.003493,neutral
...,...,...,...,...,...,...,...,...,...
25813,5,"Классное заведение,интересный интерьер и вкусн...",классный заведение интересный интерьер вкусный...,0.053413,0.870607,0.015435,0.011697,0.022987,positive
25814,1,"Залетели ночью с друзьями, разбудили работника...",залетать ночь друг разбудить работник сделать ...,0.731069,0.051855,0.122533,0.156115,0.005921,neutral
25815,5,Очень вкусная пицца. Нравится их система лояль...,очень вкусный пицца нравиться система лояльнос...,0.024433,0.956644,0.034110,0.029322,0.008587,positive
25816,5,"Пицца детям понравилась, интерьер очень интере...",пицца ребенок понравиться интерьер очень интер...,0.152042,0.943358,0.028446,0.010663,0.003085,positive


In [ ]:
# Вычисление относительных значений (в процентах)
total_mentions = topic_counts_df['Частота'].sum()
topic_counts_df['Относительная Частота (%)'] = round(((topic_counts_df['Частота'] / total_mentions) * 100),1)
top_topics1 = topic_counts_df.sort_values(by='Частота', ascending=False)

fig = px.bar(top_topics1, x='Тема', y='Относительная Частота (%)',
             title='The main aspects of catering establishments and their frequency of mentions (in percent)',
             text='Относительная Частота (%)')
fig.update_xaxes(tickangle=45, title='Topic')
fig.update_yaxes(tickangle=45, title='Frequency (%)')
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    font=dict(size=14),  # Размер шрифта для всего текста на графике
    legend=dict(
        font=dict(size=14),  # Размер шрифта для легенды
    )
)


fig.show()

In [ ]:
df['topic'] = ''

for i, text in enumerate(texts):

    review_topics = []

    for topic, keywords in topics_keywords.items():
        if any(keyword in text for keyword in keywords):

            review_topics.append(topic)

    df.at[i, 'topic'] = review_topics

In [ ]:
df

,rating,review,cleaned_review,neutral,positive,negative,skip,speech,tone,topic
0,5,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...",пить кофе рим париж вкусный капучий фундучный ...,0.160276,0.334599,0.206904,0.087574,0.007131,positive,[еда]
1,4,"Не очень удобное расположение, от метро идти м...",ужасно расположение метро идти мина быстрый ша...,0.201823,0.468801,0.090103,0.023699,0.011342,positive,"[месторасположение, еда]"
2,5,Интересное солнечное место по пути. Желтенькое...,интересный солнечный место путь желтенький люб...,0.100889,0.644235,0.036230,0.060097,0.018557,positive,"[персонал, еда]"
3,5,Уютное ламповое место. Мне всё понравилось: ра...,уютный ламповый место понравиться расположение...,0.348655,0.600198,0.038476,0.050341,0.001144,positive,"[персонал, меню, месторасположение, атмосфера,..."
4,1,Зашли единожды в это кафе. Заказали с собой ед...,зайти единожды кафе заказывать еда прощать кар...,0.569863,0.069552,0.294225,0.140346,0.003493,neutral,"[цена, атмосфера, еда]"
...,...,...,...,...,...,...,...,...,...,...
25813,5,"Классное заведение,интересный интерьер и вкусн...",классный заведение интересный интерьер вкусный...,0.053413,0.870607,0.015435,0.011697,0.022987,positive,"[персонал, еда]"
25814,1,"Залетели ночью с друзьями, разбудили работника...",залетать ночь друг разбудить работник сделать ...,0.731069,0.051855,0.122533,0.156115,0.005921,neutral,[персонал]
25815,5,Очень вкусная пицца. Нравится их система лояль...,очень вкусный пицца нравиться система лояльнос...,0.024433,0.956644,0.034110,0.029322,0.008587,positive,"[персонал, цена, маркетинг, атмосфера, еда]"
25816,5,"Пицца детям понравилась, интерьер очень интере...",пицца ребенок понравиться интерьер очень интер...,0.152042,0.943358,0.028446,0.010663,0.003085,positive,"[персонал, еда]"


In [ ]:
# Создание списка тем
topics = list(topics_keywords.keys())

rating_counts_by_topic = {topic: [] for topic in topics}
for topic in topics:

    ratings = df[df['topic'].apply(lambda x: topic in x)]['rating']

    rating_counts = ratings.value_counts(normalize=True).sort_index() * 100

    rating_counts_by_topic[topic] = rating_counts

rating_counts_df = pd.DataFrame(rating_counts_by_topic)

fig = px.bar(rating_counts_df.transpose(), x=rating_counts_df.columns, y=rating_counts_df.index,
             title='Распределение рейтингов по темам',
             labels={'value': 'Частота, %', 'index': 'Темы', 'topic': 'Тема'},
             barmode='group')

for i, topic in enumerate(topics):
    fig.update_traces(texttemplate=f'{topic}: %{{y:.1f}}%', selector=dict(name=f'{topic}'))

fig.show()

In [ ]:
df

,rating,review,cleaned_review,neutral,positive,negative,skip,speech,tone,topic
0,5,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...",пить кофе рим париж вкусный капучий фундучный ...,0.160276,0.334599,0.206904,0.087574,0.007131,positive,[еда]
1,4,"Не очень удобное расположение, от метро идти м...",ужасно расположение метро идти мина быстрый ша...,0.201823,0.468801,0.090103,0.023699,0.011342,positive,"[месторасположение, еда]"
2,5,Интересное солнечное место по пути. Желтенькое...,интересный солнечный место путь желтенький люб...,0.100889,0.644235,0.036230,0.060097,0.018557,positive,"[персонал, еда]"
3,5,Уютное ламповое место. Мне всё понравилось: ра...,уютный ламповый место понравиться расположение...,0.348655,0.600198,0.038476,0.050341,0.001144,positive,"[персонал, меню, месторасположение, атмосфера,..."
4,1,Зашли единожды в это кафе. Заказали с собой ед...,зайти единожды кафе заказывать еда прощать кар...,0.569863,0.069552,0.294225,0.140346,0.003493,neutral,"[цена, атмосфера, еда]"
...,...,...,...,...,...,...,...,...,...,...
25813,5,"Классное заведение,интересный интерьер и вкусн...",классный заведение интересный интерьер вкусный...,0.053413,0.870607,0.015435,0.011697,0.022987,positive,"[персонал, еда]"
25814,1,"Залетели ночью с друзьями, разбудили работника...",залетать ночь друг разбудить работник сделать ...,0.731069,0.051855,0.122533,0.156115,0.005921,neutral,[персонал]
25815,5,Очень вкусная пицца. Нравится их система лояль...,очень вкусный пицца нравиться система лояльнос...,0.024433,0.956644,0.034110,0.029322,0.008587,positive,"[персонал, цена, маркетинг, атмосфера, еда]"
25816,5,"Пицца детям понравилась, интерьер очень интере...",пицца ребенок понравиться интерьер очень интер...,0.152042,0.943358,0.028446,0.010663,0.003085,positive,"[персонал, еда]"


In [ ]:
df_exploded = df.explode('topic')

# Вычисляем общее количество записей для каждого рейтинга
rating_counts = df_exploded['rating'].value_counts()

topic_counts = df_exploded.groupby(['rating', 'topic']).size().unstack(fill_value=0)

# Вычисляем частоту в процентах для каждой темы
topic_freq = (topic_counts.div(rating_counts, axis=0) * 100).round(2)

fig = px.bar(topic_freq, barmode='group')
fig.update_layout(
    title='Частота тем в процентах по рейтингам',
    xaxis_title='Рейтинг',
    yaxis_title='Частота в %'
)

fig.show()

In [ ]:
import plotly.express as px

df_exploded = df.explode('topic')

df_grouped = df_exploded.groupby(['topic', 'tone']).size().reset_index(name='frequency')

total_reviews_per_topic = df_grouped.groupby('topic')['frequency'].sum()

df_grouped['frequency_percentage'] = df_grouped.apply(lambda row: (row['frequency'] / total_reviews_per_topic[row['topic']]) * 100, axis=1)

fig = px.bar(df_grouped, x='topic', y='frequency_percentage', color='tone', barmode='group',
             color_discrete_map={'positive': '#83cfb7', 'neutral': '#d5d3dd', 'negative': '#b74c4c'},  # Изменили цвета
             category_orders={'tone': ['positive', 'neutral', 'negative']},
             labels={'topic': 'Topic', 'frequency_percentage': 'Frequency (%)', 'tone': 'Sentiment'},
             title='Frequency of tones by topic, as a percentage')

fig.update_layout(
    font=dict(size=18),  # Увеличиваем размер шрифта графика
    plot_bgcolor='rgba(0,0,0,0)'  # Удаляем фон графика
)

fig.show()


In [ ]:
df

,rating,review,cleaned_review,neutral,positive,negative,skip,speech,tone,topic
0,5,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...",пить кофе рим париж вкусный капучий фундучный ...,0.160276,0.334599,0.206904,0.087574,0.007131,positive,[еда]
1,4,"Не очень удобное расположение, от метро идти м...",ужасно расположение метро идти мина быстрый ша...,0.201823,0.468801,0.090103,0.023699,0.011342,positive,"[месторасположение, еда]"
2,5,Интересное солнечное место по пути. Желтенькое...,интересный солнечный место путь желтенький люб...,0.100889,0.644235,0.036230,0.060097,0.018557,positive,"[персонал, еда]"
3,5,Уютное ламповое место. Мне всё понравилось: ра...,уютный ламповый место понравиться расположение...,0.348655,0.600198,0.038476,0.050341,0.001144,positive,"[персонал, меню, месторасположение, атмосфера,..."
4,1,Зашли единожды в это кафе. Заказали с собой ед...,зайти единожды кафе заказывать еда прощать кар...,0.569863,0.069552,0.294225,0.140346,0.003493,neutral,"[цена, атмосфера, еда]"
...,...,...,...,...,...,...,...,...,...,...
25813,5,"Классное заведение,интересный интерьер и вкусн...",классный заведение интересный интерьер вкусный...,0.053413,0.870607,0.015435,0.011697,0.022987,positive,"[персонал, еда]"
25814,1,"Залетели ночью с друзьями, разбудили работника...",залетать ночь друг разбудить работник сделать ...,0.731069,0.051855,0.122533,0.156115,0.005921,neutral,[персонал]
25815,5,Очень вкусная пицца. Нравится их система лояль...,очень вкусный пицца нравиться система лояльнос...,0.024433,0.956644,0.034110,0.029322,0.008587,positive,"[персонал, цена, маркетинг, атмосфера, еда]"
25816,5,"Пицца детям понравилась, интерьер очень интере...",пицца ребенок понравиться интерьер очень интер...,0.152042,0.943358,0.028446,0.010663,0.003085,positive,"[персонал, еда]"


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

def analyze_sentiment(text):
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = sia.polarity_scores(text)
    if sentiment_scores['compound'] >= 0.00:
        return 'positive'
    elif sentiment_scores['compound'] <= -0.00:
        return 'negative'
    else:
        return 'neutral'

def analyze_topic_sentiment_in_review_sentences(review_text, topic_keywords):
    review_sentiments = {}
    for topic, keywords in topic_keywords.items():
        topic_text = ' '.join(keywords)
        if any(keyword in review_text for keyword in keywords):
            sentences = nltk.sent_tokenize(review_text)
            positive_count = 0
            negative_count = 0
            for sentence in sentences:
                if any(keyword in sentence for keyword in keywords):
                    sentiment = analyze_sentiment(sentence)
                    if sentiment == 'positive':
                        positive_count += 1
                    elif sentiment == 'negative':
                        negative_count += 1
            if positive_count > negative_count:
                review_sentiments[topic] = 'positive'
            elif positive_count < negative_count:
                review_sentiments[topic] = 'negative'
            else:
                review_sentiments[topic] = 'neutral'
        else:
            review_sentiments[topic] = 'not_related'
    return review_sentiments

# Применяем функцию к отзывам и добавляем новые колонки с точной тональностью темы в каждом отзыве
for i, text in enumerate(texts):
    review_sentiments = analyze_topic_sentiment_in_review_sentences(text, topics_keywords)
    for topic, sentiment in review_sentiments.items():
        df.at[i, f'{topic}_sentiment'] = sentiment


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
df

,rating,review,cleaned_review,neutral,positive,negative,skip,speech,tone,topic,персонал_sentiment,меню_sentiment,цена_sentiment,маркетинг_sentiment,санитария_sentiment,месторасположение_sentiment,атмосфера_sentiment,еда_sentiment
0,5,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...",пить кофе рим париж вкусный капучий фундучный ...,0.160276,0.334599,0.206904,0.087574,0.007131,positive,[еда],not_related,not_related,not_related,not_related,not_related,not_related,not_related,positive
1,4,"Не очень удобное расположение, от метро идти м...",ужасно расположение метро идти мина быстрый ша...,0.201823,0.468801,0.090103,0.023699,0.011342,positive,"[месторасположение, еда]",not_related,not_related,not_related,not_related,not_related,positive,not_related,positive
2,5,Интересное солнечное место по пути. Желтенькое...,интересный солнечный место путь желтенький люб...,0.100889,0.644235,0.036230,0.060097,0.018557,positive,"[персонал, еда]",positive,not_related,not_related,not_related,not_related,not_related,not_related,positive
3,5,Уютное ламповое место. Мне всё понравилось: ра...,уютный ламповый место понравиться расположение...,0.348655,0.600198,0.038476,0.050341,0.001144,positive,"[персонал, меню, месторасположение, атмосфера,...",positive,positive,not_related,not_related,not_related,positive,positive,positive
4,1,Зашли единожды в это кафе. Заказали с собой ед...,зайти единожды кафе заказывать еда прощать кар...,0.569863,0.069552,0.294225,0.140346,0.003493,neutral,"[цена, атмосфера, еда]",not_related,not_related,positive,not_related,not_related,not_related,positive,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25813,5,"Классное заведение,интересный интерьер и вкусн...",классный заведение интересный интерьер вкусный...,0.053413,0.870607,0.015435,0.011697,0.022987,positive,"[персонал, еда]",positive,not_related,not_related,not_related,not_related,not_related,not_related,positive
25814,1,"Залетели ночью с друзьями, разбудили работника...",залетать ночь друг разбудить работник сделать ...,0.731069,0.051855,0.122533,0.156115,0.005921,neutral,[персонал],positive,not_related,not_related,not_related,not_related,not_related,not_related,not_related
25815,5,Очень вкусная пицца. Нравится их система лояль...,очень вкусный пицца нравиться система лояльнос...,0.024433,0.956644,0.034110,0.029322,0.008587,positive,"[персонал, цена, маркетинг, атмосфера, еда]",positive,not_related,positive,positive,not_related,not_related,positive,positive
25816,5,"Пицца детям понравилась, интерьер очень интере...",пицца ребенок понравиться интерьер очень интер...,0.152042,0.943358,0.028446,0.010663,0.003085,positive,"[персонал, еда]",positive,not_related,not_related,not_related,not_related,not_related,not_related,positive


In [ ]:
df['еда_sentiment'].value_counts()

еда_sentiment
positive       19786
not_related     6032
Name: count, dtype: int64

In [ ]:
df['персонал_sentiment'].value_counts()

персонал_sentiment
positive       13123
not_related    12695
Name: count, dtype: int64

In [ ]:
df['topic'].apply(lambda x: x.count('еда')).sum()

19786

In [ ]:
import re
import collections

# Создание словаря для хранения пар слов для каждого ключевого слова в каждой теме
keyword_pairs = {topic: collections.defaultdict(list) for topic in topics_keywords.keys()}

# Итерация по всем текстам каждой темы
for topic, texts in topic_texts.items():
    for text in texts:
        # Проход по каждому ключевому слову темы
        for keyword in topics_keywords[topic]:
            # Находим индексы всех вхождений ключевого слова в текст
            indices = [m.start() for m in re.finditer(keyword, text)]
            # Для каждого вхождения находим соседние слова
            for index in indices:
                # Индексы начала и конца ключевого слова
                start = max(0, index - 50)  # Начало интервала 50 символов до ключевого слова
                end = min(len(text), index + 50 + len(keyword))  # Конец интервала 50 символов после ключевого слова
                # Извлечение текста вокруг ключевого слова
                surrounding_text = text[start:end]
                # Разбивка текста на слова
                words = surrounding_text.split()
                # Поиск пар слов, содержащих ключевое слово и прилагательное
                for i in range(len(words) - 1):
                    if words[i] == keyword and 'ADJF' in morph.parse(words[i + 1])[0].tag:
                        # Добавление пар слов в словарь
                        keyword_pairs[topic][keyword].append((words[i], words[i + 1]))

# Вывод пар слов для каждого ключевого слова в каждой теме
for topic, pairs in keyword_pairs.items():
    print("Тема:", topic)
    for keyword, word_pairs in pairs.items():
        print("Ключевое слово:", keyword)
        print("Пары слов:", word_pairs)
        print()



Тема: персонал
Ключевое слово: персонал
Пары слов: [('персонал', 'внимательный'), ('персонал', 'высококвалифицированный'), ('персонал', 'отличный'), ('персонал', 'дружелюбный'), ('персонал', 'приветливый'), ('персонал', 'отличный'), ('персонал', 'приветливый'), ('персонал', 'большой'), ('персонал', 'приятный'), ('персонал', 'приветливый'), ('персонал', 'вкусный'), ('персонал', 'приветливый'), ('персонал', 'просторный'), ('персонал', 'доброжелательный'), ('персонал', 'замечательный'), ('персонал', 'приветливый'), ('персонал', 'оперативный'), ('персонал', 'который'), ('персонал', 'вежливый'), ('персонал', 'турецкий'), ('персонал', 'приятный'), ('персонал', 'вкусный'), ('персонал', 'отличный'), ('персонал', 'хороший'), ('персонал', 'доброжелательный'), ('персонал', 'вежливый'), ('персонал', 'праздничный'), ('персонал', 'приятный'), ('персонал', 'вежливый'), ('персонал', 'хороший'), ('персонал', 'внимательный'), ('персонал', 'грамотный'), ('персонал', 'живой'), ('персонал', 'быстрый'), ('п

In [ ]:
from collections import Counter

# Создание словаря для хранения самых популярных пар слов для каждого ключевого слова в каждой теме
top_keyword_pairs = {topic: {} for topic in topics_keywords.keys()}

# Итерация по всем парам слов для каждой темы
for topic, pairs in keyword_pairs.items():
    top_keyword_pairs[topic] = {keyword: Counter(pairs[keyword]).most_common(5) for keyword in pairs}

# Вывод самых популярных пар слов для каждого ключевого слова в каждой теме
for topic, keyword_pairs in top_keyword_pairs.items():
    print("Тема:", topic)
    for keyword, pairs in keyword_pairs.items():
        print("Ключевое слово:", keyword)
        print("Самые популярные пары слов:", pairs)
        print()


Тема: персонал
Ключевое слово: персонал
Самые популярные пары слов: [(('персонал', 'вежливый'), 408), (('персонал', 'приветливый'), 231), (('персонал', 'вкусный'), 194), (('персонал', 'хороший'), 123), (('персонал', 'приятный'), 106)]

Ключевое слово: обслуживание
Самые популярные пары слов: [(('обслуживание', 'высокий'), 185), (('обслуживание', 'быстрый'), 136), (('обслуживание', 'хороший'), 129), (('обслуживание', 'вкусный'), 103), (('обслуживание', 'отличный'), 93)]

Ключевое слово: официант
Самые популярные пары слов: [(('официант', 'вежливый'), 119), (('официант', 'который'), 82), (('официант', 'приветливый'), 60), (('официант', 'весь'), 47), (('официант', 'внимательный'), 41)]

Ключевое слово: бармен
Самые популярные пары слов: [(('бармен', 'который'), 15), (('бармен', 'хороший'), 8), (('бармен', 'приятный'), 7), (('бармен', 'отдельный'), 7), (('бармен', 'отличный'), 6)]

Ключевое слово: сервис
Самые популярные пары слов: [(('сервис', 'хороший'), 38), (('сервис', 'высокий'), 29),

In [ ]:
# Создание списка для хранения данных
data = []

# Итерация по всем парам слов для каждой темы
for topic, keyword_pairs in top_keyword_pairs.items():
    for keyword, pairs in keyword_pairs.items():
        for pair, count in pairs:
            data.append([topic, keyword, pair, count])

# Создание DataFrame из списка данных
df_pairs = pd.DataFrame(data, columns=['Тема', 'Ключевое слово', 'Пара слов', 'Частота'])

# Вывод DataFrame
df_pairs

,Тема,Ключевое слово,Пара слов,Частота
0,персонал,персонал,"(персонал, вежливый)",408
1,персонал,персонал,"(персонал, приветливый)",231
2,персонал,персонал,"(персонал, вкусный)",194
3,персонал,персонал,"(персонал, хороший)",123
4,персонал,персонал,"(персонал, приятный)",106
...,...,...,...,...
331,еда,божественный,"(божественный, самый)",1
332,еда,божественный,"(божественный, общий)",1
333,еда,божественный,"(божественный, отдельный)",1
334,еда,божественно,"(божественно, вкусный)",2


In [ ]:
# Создание списка для хранения данных
data = []

# Итерация по каждой теме
for topic, keyword_pairs in top_keyword_pairs.items():
    # Находим самое популярное ключевое слово и его пары
    most_popular_keyword = max(keyword_pairs, key=lambda x: sum(pair[1] for pair in keyword_pairs[x]))
    most_popular_pairs = keyword_pairs[most_popular_keyword][:3]

    # Добавляем информацию в список данных
    for pair, count in most_popular_pairs:
        data.append([topic, most_popular_keyword, pair, count])

# Создание DataFrame из списка данных
df_popular_pairs = pd.DataFrame(data, columns=['Тема', 'Ключевое слово', 'Пара слов', 'Частота'])

df_popular_pairs

,Тема,Ключевое слово,Пара слов,Частота
0,персонал,персонал,"(персонал, вежливый)",408
1,персонал,персонал,"(персонал, приветливый)",231
2,персонал,персонал,"(персонал, вкусный)",194
3,меню,меню,"(меню, разнообразный)",38
4,меню,меню,"(меню, вкусный)",25
5,меню,меню,"(меню, хороший)",17
6,цена,цена,"(цена, приемлемый)",57
7,цена,цена,"(цена, средний)",38
8,цена,цена,"(цена, адекватный)",33
9,маркетинг,акция,"(акция, который)",2


In [ ]:
df

,rating,review,cleaned_review,neutral,positive,negative,skip,speech,tone,topic,персонал_sentiment,меню_sentiment,цена_sentiment,маркетинг_sentiment,санитария_sentiment,месторасположение_sentiment,атмосфера_sentiment,еда_sentiment
0,5,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...",пить кофе рим париж вкусный капучий фундучный ...,0.160276,0.334599,0.206904,0.087574,0.007131,positive,[еда],not_related,not_related,not_related,not_related,not_related,not_related,not_related,positive
1,4,"Не очень удобное расположение, от метро идти м...",ужасно расположение метро идти мина быстрый ша...,0.201823,0.468801,0.090103,0.023699,0.011342,positive,"[месторасположение, еда]",not_related,not_related,not_related,not_related,not_related,positive,not_related,positive
2,5,Интересное солнечное место по пути. Желтенькое...,интересный солнечный место путь желтенький люб...,0.100889,0.644235,0.036230,0.060097,0.018557,positive,"[персонал, еда]",positive,not_related,not_related,not_related,not_related,not_related,not_related,positive
3,5,Уютное ламповое место. Мне всё понравилось: ра...,уютный ламповый место понравиться расположение...,0.348655,0.600198,0.038476,0.050341,0.001144,positive,"[персонал, меню, месторасположение, атмосфера,...",positive,positive,not_related,not_related,not_related,positive,positive,positive
4,1,Зашли единожды в это кафе. Заказали с собой ед...,зайти единожды кафе заказывать еда прощать кар...,0.569863,0.069552,0.294225,0.140346,0.003493,neutral,"[цена, атмосфера, еда]",not_related,not_related,positive,not_related,not_related,not_related,positive,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25813,5,"Классное заведение,интересный интерьер и вкусн...",классный заведение интересный интерьер вкусный...,0.053413,0.870607,0.015435,0.011697,0.022987,positive,"[персонал, еда]",positive,not_related,not_related,not_related,not_related,not_related,not_related,positive
25814,1,"Залетели ночью с друзьями, разбудили работника...",залетать ночь друг разбудить работник сделать ...,0.731069,0.051855,0.122533,0.156115,0.005921,neutral,[персонал],positive,not_related,not_related,not_related,not_related,not_related,not_related,not_related
25815,5,Очень вкусная пицца. Нравится их система лояль...,очень вкусный пицца нравиться система лояльнос...,0.024433,0.956644,0.034110,0.029322,0.008587,positive,"[персонал, цена, маркетинг, атмосфера, еда]",positive,not_related,positive,positive,not_related,not_related,positive,positive
25816,5,"Пицца детям понравилась, интерьер очень интере...",пицца ребенок понравиться интерьер очень интер...,0.152042,0.943358,0.028446,0.010663,0.003085,positive,"[персонал, еда]",positive,not_related,not_related,not_related,not_related,not_related,not_related,positive


In [ ]:
# Подсчитать общее количество отзывов по каждому рейтингу
total_reviews_per_rating = df.groupby('rating').size().reset_index(name='total_reviews')

# Развернуть списки тем и тональностей
df_expanded = df.explode('topic').explode('tone')

# Подсчитать количество отзывов по каждой теме, рейтингу и тональности
topic_reviews_count = df_expanded.groupby(['rating', 'topic', 'tone']).size().reset_index(name='topic_reviews')

# Объединить данные
merged_df = pd.merge(topic_reviews_count, total_reviews_per_rating, on='rating')

# Рассчитать частоту тем и тональности в процентах
merged_df['frequency'] = (merged_df['topic_reviews'] / merged_df['total_reviews']) * 100

# Построить графики для каждой темы
fig = px.bar(merged_df, x='rating', y='frequency', color='tone', barmode='group',
             facet_col='topic', facet_col_wrap=2, category_orders={'tone': ['positive', 'negative', 'neutral']},
             labels={'frequency': 'Частота, %', 'rating': 'Рейтинг'},
             title='Частота тем и тональности в процентах по рейтингу')
fig.update_layout(height=1000, width=1000)  # Увеличиваем размер графика
fig.show()

In [ ]:
df1 = df.iloc[:, :-9]

In [ ]:
import pandas as pd
from transformers import pipeline

# Предположим, что у вас уже есть DataFrame 'df' с колонками 'cleaned_reviews' и 'topic'
topics_keywords = {
    'персонал': ['официанты', 'официант', 'работник', 'работники', 'персонал', 'кассир', 'администратор', 'хостес', 'сотрудники', 'бармен', 'бармэн', 'бариста', 'сомелье', 'обслуживание', 'сервис', 'отношение'],
    'меню': ['ассортимент', 'меню', 'выбор', 'карта', 'разнообразие', 'разнообразный'],
    'цена': ['стоимость', 'цена', 'дорого', 'дешево', 'дёшево', 'бюджетно', 'дешевый', 'бюджетный', 'дорогой'],
    'маркетинг': ['реклама', 'маркетинг', 'продвижение', 'бренд', 'акция', 'известный', 'лояльность', 'раскрученный', 'популярный', 'пиар', 'распиаренный', 'блогер', 'малоизвестный'],
    'санитария': ['санитария', 'гигиена', 'чистота', 'уборщик', 'чистый', 'грязный', 'уборка', 'убрано', 'грязно', 'пыльно', 'убрано', 'тараканы', 'волос', 'убранный', 'налет', 'мытое', 'мытый', 'немытое', 'мухи'],
    'месторасположение': ['локация', 'месторасположение', 'расположение', 'доступность', 'достопримечательность', 'удобный', 'дорога', 'рядом', 'далеко', 'неудобно', 'удобно', 'близко', 'близкий', 'далекий'],
    'атмосфера': ['обстановка', 'концепция', 'стиль', 'атмосфера', 'тематика', 'уют', 'уютный', 'вайб', 'атмосферный', 'домашний', 'дом', 'неуютный', 'неуютно', 'приятный', 'комфортный', 'комфортно', 'комфорт', 'неприятный'],
    'еда': ['сочетание', 'божественно', 'еда', 'блюдо', 'кухня', 'вкус', 'повар', 'качество', 'свежесть', 'продукт', 'качественный', 'свежий', 'вкусно', 'вкусный', 'невкусный', 'божественный']
}

# Загрузка предобученной модели анализа тональности
sentiment_analyzer = pipeline("sentiment-analysis")

def get_sentiment(text):
    results = sentiment_analyzer(text)
    if results[0]['label'] == 'POSITIVE':
        return 'positive'
    elif results[0]['label'] == 'NEGATIVE':
        return 'negative'
    else:
        return 'neutral'

def analyze_topics_sentiment(row):
    review = row['cleaned_review']
    topics = row['topic']
    sentiments = {}

    for topic in topics:
        topic_keywords = topics_keywords[topic]
        topic_text = " ".join([word for word in review.split() if word in topic_keywords])
        if topic_text:
            sentiments[topic] = get_sentiment(topic_text)
        else:
            sentiments[topic] = 'neutral'  # Если ключевые слова не найдены, считаем тональность нейтральной

    return sentiments

df['topic_sentiments'] = df.apply(analyze_topics_sentiment, axis=1)



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
df

,rating,review,cleaned_review,neutral,positive,negative,skip,speech,tone,topic,персонал_sentiment,меню_sentiment,цена_sentiment,маркетинг_sentiment,санитария_sentiment,месторасположение_sentiment,атмосфера_sentiment,еда_sentiment,topic_sentiments
0,5,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...",пить кофе рим париж вкусный капучий фундучный ...,0.160276,0.334599,0.206904,0.087574,0.007131,positive,[еда],not_related,not_related,not_related,not_related,not_related,not_related,not_related,positive,{'еда': 'positive'}
1,4,"Не очень удобное расположение, от метро идти м...",ужасно расположение метро идти мина быстрый ша...,0.201823,0.468801,0.090103,0.023699,0.011342,positive,"[месторасположение, еда]",not_related,not_related,not_related,not_related,not_related,positive,not_related,positive,"{'месторасположение': 'positive', 'еда': 'posi..."
2,5,Интересное солнечное место по пути. Желтенькое...,интересный солнечный место путь желтенький люб...,0.100889,0.644235,0.036230,0.060097,0.018557,positive,"[персонал, еда]",positive,not_related,not_related,not_related,not_related,not_related,not_related,positive,"{'персонал': 'positive', 'еда': 'positive'}"
3,5,Уютное ламповое место. Мне всё понравилось: ра...,уютный ламповый место понравиться расположение...,0.348655,0.600198,0.038476,0.050341,0.001144,positive,"[персонал, меню, месторасположение, атмосфера,...",positive,positive,not_related,not_related,not_related,positive,positive,positive,"{'персонал': 'positive', 'меню': 'positive', '..."
4,1,Зашли единожды в это кафе. Заказали с собой ед...,зайти единожды кафе заказывать еда прощать кар...,0.569863,0.069552,0.294225,0.140346,0.003493,neutral,"[цена, атмосфера, еда]",not_related,not_related,positive,not_related,not_related,not_related,positive,positive,"{'цена': 'positive', 'атмосфера': 'neutral', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25813,5,"Классное заведение,интересный интерьер и вкусн...",классный заведение интересный интерьер вкусный...,0.053413,0.870607,0.015435,0.011697,0.022987,positive,"[персонал, еда]",positive,not_related,not_related,not_related,not_related,not_related,not_related,positive,"{'персонал': 'positive', 'еда': 'positive'}"
25814,1,"Залетели ночью с друзьями, разбудили работника...",залетать ночь друг разбудить работник сделать ...,0.731069,0.051855,0.122533,0.156115,0.005921,neutral,[персонал],positive,not_related,not_related,not_related,not_related,not_related,not_related,not_related,{'персонал': 'positive'}
25815,5,Очень вкусная пицца. Нравится их система лояль...,очень вкусный пицца нравиться система лояльнос...,0.024433,0.956644,0.034110,0.029322,0.008587,positive,"[персонал, цена, маркетинг, атмосфера, еда]",positive,not_related,positive,positive,not_related,not_related,positive,positive,"{'персонал': 'positive', 'цена': 'positive', '..."
25816,5,"Пицца детям понравилась, интерьер очень интере...",пицца ребенок понравиться интерьер очень интер...,0.152042,0.943358,0.028446,0.010663,0.003085,positive,"[персонал, еда]",positive,not_related,not_related,not_related,not_related,not_related,not_related,positive,"{'персонал': 'positive', 'еда': 'positive'}"


In [ ]:
data = pd.read_excel('/content/dfffff.xlsx')

NameError: name 'pd' is not defined

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
from transformers import pipeline
from nltk.tokenize import word_tokenize
import nltk
!pip install pymorphy2
import pymorphy2

# Загрузка данных nltk
nltk.download('punkt')

# Инициализация предобученной модели для анализа тональности
sentiment_pipeline = pipeline('sentiment-analysis')

# Инициализация морфологического анализатора для русского языка
morph = pymorphy2.MorphAnalyzer()

# Функция для определения тональности слова с использованием кэширования
cache = {}

def get_word_sentiment(word):
    if word in cache:
        return cache[word]

    result = sentiment_pipeline(word)
    sentiment = result[0]['label']
    if sentiment == 'POSITIVE':
        cache[word] = 'positive'
    elif sentiment == 'NEGATIVE':
        cache[word] = 'negative'
    else:
        cache[word] = 'neutral'

    return cache[word]

def determine_sentiment(review, topic_keywords):
    sentiment = {topic: 'neutral' for topic in topic_keywords}

    for i, word in enumerate(review):
        parsed_word = morph.parse(word)[0]
        for topic, keywords in topic_keywords.items():
            if word in keywords:
                prev_sentiment = 'neutral'
                next_sentiment = 'neutral'

                # Проверка прилагательных перед ключевым словом
                if i > 0:
                    prev_word = review[i-1]
                    prev_parsed = morph.parse(prev_word)[0]
                    if 'ADJF' in prev_parsed.tag:  # Прилагательные в русском языке
                        prev_sentiment = get_word_sentiment(prev_word)

                # Проверка прилагательных после ключевого слова
                if i < len(review) - 1:
                    next_word = review[i+1]
                    next_parsed = morph.parse(next_word)[0]
                    if 'ADJF' in next_parsed.tag:  # Прилагательные в русском языке
                        next_sentiment = get_word_sentiment(next_word)

                # Определение итоговой тональности для темы
                if prev_sentiment == 'negative' or next_sentiment == 'negative':
                    sentiment[topic] = 'negative'
                elif prev_sentiment == 'positive' or next_sentiment == 'positive':
                    sentiment[topic] = 'positive'

    return sentiment

# Задание ключевых слов для каждой темы
topics_keywords = {
    'персонал': ['официанты', 'официант', 'работник', 'работники', 'персонал', 'кассир', 'администратор', 'хостес', 'сотрудники', 'бармен', 'бармэн', 'бариста', 'сомелье', 'обслуживание', 'сервис', 'отношение'],
    'меню': ['ассортимент', 'меню', 'выбор', 'карта', 'разнообразие', 'разнообразный'],
    'цена': ['стоимость', 'цена', 'дорого', 'дешево', 'дёшево', 'бюджетно', 'дешевый', 'бюджетный', 'дорогой'],
    'маркетинг': ['реклама', 'маркетинг', 'продвижение', 'бренд', 'акция', 'известный', 'лояльность', 'раскрученный', 'популярный', 'пиар', 'распиаренный', 'блогер', 'малоизвестный'],
    'санитария': ['санитария', 'гигиена', 'чистота', 'уборщик', 'чистый', 'грязный', 'уборка', 'убрано', 'грязно', 'пыльно', 'убрано', 'тараканы', 'волос', 'убранный', 'налет', 'мытое', 'мытый', 'немытое', 'мухи'],
    'месторасположение': ['локация', 'месторасположение', 'расположение', 'доступность', 'достопримечательность', 'удобный', 'дорога', 'рядом', 'далеко', 'неудобно', 'удобно', 'близко', 'близкий', 'далекий'],
    'атмосфера': ['обстановка', 'концепция', 'стиль', 'атмосфера', 'тематика', 'уют', 'уютный', 'вайб', 'атмосферный', 'домашний', 'дом', 'неуютный', 'неуютно', 'приятный', 'комфортный', 'комфортно', 'комфорт', 'неприятный'],
    'еда': ['сочетание', 'божественно', 'еда', 'блюдо', 'кухня', 'вкус', 'повар', 'качество', 'свежесть', 'продукт', 'качественный', 'свежий', 'вкусно', 'вкусный', 'невкусный', 'божественный']
}

# Применение функции determine_sentiment для каждого отзыва и сохранение результатов в новых столбцах
for i, row in data.iterrows():
    review = word_tokenize(row['cleaned_review'])
    sentiment = determine_sentiment(review, topics_keywords)

    for topic, value in sentiment.items():
        data.at[i, f'{topic}_sentiment'] = value

# Вывод результатов
data[['review', 'cleaned_review'] + [f'{topic}_sentiment' for topic in topics_keywords.keys()]]



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 46.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=54b55b77faaeddc4541189c6b9443cb4070bff0c41f15afcd4e7cffc6187cb11
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.



,review,cleaned_review,персонал_sentiment,меню_sentiment,цена_sentiment,маркетинг_sentiment,санитария_sentiment,месторасположение_sentiment,атмосфера_sentiment,еда_sentiment
0,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...",пить кофе рим париж вкусный капучий фундучный ...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
1,"Не очень удобное расположение, от метро идти м...",ужасно расположение метро идти мина быстрый ша...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,positive
2,Интересное солнечное место по пути. Желтенькое...,интересный солнечный место путь желтенький люб...,positive,neutral,neutral,neutral,neutral,neutral,neutral,positive
3,Уютное ламповое место. Мне всё понравилось: ра...,уютный ламповый место понравиться расположение...,positive,neutral,neutral,neutral,neutral,neutral,positive,neutral
4,Зашли единожды в это кафе. Заказали с собой ед...,зайти единожды кафе заказывать еда прощать кар...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,positive
...,...,...,...,...,...,...,...,...,...,...
25813,"Классное заведение,интересный интерьер и вкусн...",классный заведение интересный интерьер вкусный...,positive,neutral,neutral,neutral,neutral,neutral,neutral,positive
25814,"Залетели ночью с друзьями, разбудили работника...",залетать ночь друг разбудить работник сделать ...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,neutral
25815,Очень вкусная пицца. Нравится их система лояль...,очень вкусный пицца нравиться система лояльнос...,positive,neutral,positive,positive,neutral,neutral,neutral,neutral
25816,"Пицца детям понравилась, интерьер очень интере...",пицца ребенок понравиться интерьер очень интер...,positive,neutral,neutral,neutral,neutral,neutral,neutral,positive


In [ ]:
def determine_sentiment(review, topic_keywords):
    sentiment = {}

    for topic, keywords in topic_keywords.items():
        sentiment[topic] = 'neutral'

    for i, word in enumerate(review):
        for topic, keywords in topic_keywords.items():
            if word in keywords:
                # Проверка наличия отрицательных слов в предложении перед ключевым словом
                if i > 0 and review[i-1] in ['не', 'ни']:
                    sentiment[topic] = 'negative'
                else:
                    sentiment[topic] = 'positive'
                # Проверка наличия отрицательных слов в предложении после ключевого слова
                if i < len(review) - 1 and review[i+1] in ['не', 'ни']:
                    sentiment[topic] = 'negative'

    return sentiment



# Задание ключевых слов для каждой темы
topics_keywords = {
    'персонал': ['официанты', 'официант', 'работник', 'работники', 'персонал', 'кассир', 'администратор', 'хостес', 'сотрудники', 'бармен', 'бармэн', 'бариста', 'сомелье', 'обслуживание', 'сервис', 'отношение'],
    'меню': ['ассортимент', 'меню', 'выбор', 'карта', 'разнообразие', 'разнообразный'],
    'цена': ['стоимость', 'цена', 'дорого', 'дешево', 'дёшево', 'бюджетно', 'дешевый', 'бюджетный', 'дорогой'],
    'маркетинг': ['реклама', 'маркетинг', 'продвижение', 'бренд', 'акция', 'известный', 'лояльность', 'раскрученный', 'популярный', 'пиар', 'распиаренный', 'блогер', 'малоизвесный'],
    'санитария': ['санитария', 'гигиена', 'чистота', 'уборщик', 'чистый', 'грязный', 'уборка', 'убрано', 'грязно', 'пыльно', 'убрано', 'тараканы', 'волос', 'убранный', 'налет', 'мытое', 'мытый', 'немытое', 'мухи'],
    'месторасположение': ['локация', 'месторасположение', 'расположение', 'доступность', 'достопримечательность', 'удобный', 'дорога', 'рядом', 'далеко', 'неудобно', 'удобно', 'близко', 'близкий', 'далекий'],
    'атмосфера': ['обстановка', 'концепция', 'стиль', 'атмосфера', 'тематика', 'уют', 'уютный', 'вайб', 'атмосферный', 'домашний', 'дом', 'неуютный', 'неуютно', 'приятный', 'комфортный', 'комфортно', 'комфорт', 'неприятный'],
    'еда': ['сочетание', 'божественно', 'еда', 'блюдо', 'кухня', 'вкус', 'повар', 'качество', 'свежесть', 'продукт', 'качественный', 'свежий', 'вкусно', 'вкусный', 'невкусный', 'божественный']
}


# Применение функции determine_sentiment для каждого отзыва и сохранение результатов в новых столбцах
for i, row in data.iterrows():
    review = row['cleaned_review'].split()
    sentiment = determine_sentiment(review, topics_keywords)

    for topic, value in sentiment.items():
        data.at[i, f'{topic}_sentiment'] = value

# Вывод результатов
data[['review', 'cleaned_review'] + [f'{topic}_sentiment' for topic in topics_keywords.keys()]]

,review,cleaned_review,персонал_sentiment,меню_sentiment,цена_sentiment,маркетинг_sentiment,санитария_sentiment,месторасположение_sentiment,атмосфера_sentiment,еда_sentiment
0,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...",пить кофе рим париж вкусный капучий фундучный ...,neutral,neutral,neutral,neutral,neutral,neutral,neutral,positive
1,"Не очень удобное расположение, от метро идти м...",ужасно расположение метро идти мина быстрый ша...,neutral,neutral,neutral,neutral,neutral,positive,neutral,positive
2,Интересное солнечное место по пути. Желтенькое...,интересный солнечный место путь желтенький люб...,positive,neutral,neutral,neutral,neutral,neutral,neutral,positive
3,Уютное ламповое место. Мне всё понравилось: ра...,уютный ламповый место понравиться расположение...,positive,positive,neutral,neutral,neutral,positive,positive,positive
4,Зашли единожды в это кафе. Заказали с собой ед...,зайти единожды кафе заказывать еда прощать кар...,neutral,neutral,positive,neutral,neutral,neutral,neutral,positive
...,...,...,...,...,...,...,...,...,...,...
25813,"Классное заведение,интересный интерьер и вкусн...",классный заведение интересный интерьер вкусный...,positive,neutral,neutral,neutral,neutral,neutral,neutral,positive
25814,"Залетели ночью с друзьями, разбудили работника...",залетать ночь друг разбудить работник сделать ...,positive,neutral,neutral,neutral,neutral,neutral,neutral,neutral
25815,Очень вкусная пицца. Нравится их система лояль...,очень вкусный пицца нравиться система лояльнос...,positive,neutral,positive,positive,neutral,neutral,positive,positive
25816,"Пицца детям понравилась, интерьер очень интере...",пицца ребенок понравиться интерьер очень интер...,positive,neutral,neutral,neutral,neutral,neutral,neutral,positive


In [ ]:
import plotly.express as px
import pandas as pd

# Предполагаем, что у вас есть DataFrame data с колонками 'review', 'cleaned_review' и столбцами для каждой темы с суффиксом '_sentiment'

# Создание временного DataFrame для подсчета частоты сентиментов для каждой темы
sentiment_counts = {'topic': [], 'positive': [], 'negative': [], 'neutral': []}

for topic in topics_keywords.keys():
    # Подсчет количества каждого сентимента для данной темы
    sentiment_counts['topic'].append(topic)
    sentiment_counts['positive'].append((data[f'{topic}_sentiment'] == 'positive').mean() * 100)
    sentiment_counts['negative'].append((data[f'{topic}_sentiment'] == 'negative').mean() * 100)
    sentiment_counts['neutral'].append((data[f'{topic}_sentiment'] == 'neutral').mean() * 100)

# Создание DataFrame из словаря
sentiment_df = pd.DataFrame(sentiment_counts)

# Построение stacked bar графика
fig = px.bar(sentiment_df, x='topic', y=['positive', 'negative', 'neutral'],
             title='The frequency of sentiment on each topic',
             labels={'topic': 'Тема', 'value': 'Проценты', 'variable': 'Сентимент'},
             barmode='stack')

fig.show()


In [ ]:
import plotly.express as px
import pandas as pd

# Предполагаем, что у вас есть DataFrame data с колонками 'review', 'cleaned_review' и столбцами для каждой темы с суффиксом '_sentiment'

# Создание временного DataFrame для подсчета частоты сентиментов для каждой темы
sentiment_counts = {'topic': [], 'positive': [], 'negative': []}

for topic in topics_keywords.keys():
    # Подсчет количества каждого сентимента для данной темы
    sentiment_counts['topic'].append(topic)
    sentiment_counts['positive'].append((data[f'{topic}_sentiment'] == 'positive').mean() * 100)
    sentiment_counts['negative'].append((data[f'{topic}_sentiment'] == 'negative').mean() * 100)

# Создание DataFrame из словаря
sentiment_df = pd.DataFrame(sentiment_counts)

# Построение stacked bar графика
fig = px.bar(sentiment_df, x='topic', y=['positive', 'negative'],
             title='The frequency of sentiment on each topic',
             labels={'topic': 'Тема', 'value': 'Проценты', 'variable': 'Сентимент'},
             barmode='stack')

fig.show()

In [ ]:
import plotly.express as px
import pandas as pd

# Предполагаем, что у вас есть DataFrame data с колонками 'review', 'cleaned_review' и столбцами для каждой темы с суффиксом '_sentiment'

# Создание временного DataFrame для подсчета частоты сентиментов для каждой темы
sentiment_counts = {'topic': [], 'positive': [], 'negative': []}

for topic in topics_keywords.keys():
    # Подсчет количества каждого сентимента для данной темы
    pos_count = (data[f'{topic}_sentiment'] == 'positive').sum()
    neg_count = (data[f'{topic}_sentiment'] == 'negative').sum()
    total = pos_count + neg_count

    # Нормализация значений positive и negative до 100%
    if total > 0:
        sentiment_counts['topic'].append(topic)
        sentiment_counts['positive'].append((pos_count / total) * 100)
        sentiment_counts['negative'].append((neg_count / total) * 100)

# Создание DataFrame из словаря
sentiment_df = pd.DataFrame(sentiment_counts)

# Построение stacked bar графика
fig = px.bar(sentiment_df, x='topic', y=['positive', 'negative'],
             title='The frequency of sentiment on each topic (without neutral sentiment)',
             labels={'topic': 'Тема', 'value': 'Проценты', 'variable': 'Сентимент'},
             barmode='stack')

fig.show()